In [57]:
import os
import shutil
import random
import keras
import tensorflow

In [58]:
import tensorflow as tf

# Load all Models
model_large = keras.models.load_model("Trained_Mobile_Large_Model")
model_small = keras.models.load_model("Trained_Mobile_Small_Model")
model_custom = keras.models.load_model("Trained_Mobile_Custom_Five_Model")

In [59]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

test_path = 'data/Farm_Fresh_Dataset_PNG/'
def prepare_image(file_path):
    """
    Preprocess the input images
    
    """
    image = tf.keras.utils.load_img(file_path, target_size=(224, 224))
    image_array = tf.keras.utils.img_to_array(image)
    image_array_expand_dims = np.expand_dims(image_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(image_array_expand_dims)

In [60]:
from IPython.display import Image

In [61]:
def get_prediction_confidence(prediction, classes):
    """
    Get the models prediction likelyhood on each class
    
    """
    index = 0
    read_prediction = [None] * len(classes)
    
    for confidence in prediction:
        read_prediction[index] = (confidence, classes[index])
        index = index + 1
    return sorted(read_prediction, reverse=True)

In [117]:
import random
import time

def random_image_selector():
    """
    Use random to select a directory from 0 to 3 and then select an image from 0 to 99
    
    """
    image_class = ""
    target_directory = random.randint(0, 3)
    target_image = random.randint(0, 99)
    if target_directory == 0:
        image_class = 'Asparagus'
    elif target_directory == 1:
        image_class = 'BrusselSprouts'
    elif target_directory == 2:
        image_class = 'Celery'
    elif target_directory == 3:
        image_class = 'SliverBeet'
    else:
        print("error")
    return test_path + str(target_directory) + "/" + str(target_image) + ".PNG", image_class
    
    
def predict_image(model_type, image_path):
    """
    Get predictions from the model on given image
    
    """
    preprocessed_image = prepare_image(image_path)
    
    start_time = time.time()
    predictions = model_type.predict(preprocessed_image)
    total_time = time.time() - start_time
    classed_predictions = get_prediction_confidence(
        predictions[0], 
        classes=['Asparagus', 'BrusselSprouts', 'Celery', 'SliverBeet']
    )
    
    return [classed_predictions, total_time]


In [120]:
import tkinter as tk
from tkinter import *
from PIL import ImageTk, Image
import PIL
from tkinter import messagebox

global_image_path = ''

def generate_prediction_text(prediction):
    """
    Get the prediction text
    
    """
    return 'Predicts ' + prediction[1] + ' at ' + str(prediction[0] * 100) + '%'


def generate_prediction_time(time):
    """
    Get prediction time
    
    """
    return "Prediction time took " + str("%.3f" % round(time, 3) + " seconds")
    

def update_canvas(canvas, mobile_id_list, model_type):
    """
    update_canvas() takes the container of an image on a canvas and update
    it to a new image
    
    """
    predict_1, predict_2, predict_3, predict_4, predict_time = mobile_id_list
    prediction, total_time = predict_image(model_type, global_image_path)
    
    canvas.itemconfig(predict_1, text=generate_prediction_text(prediction[0]))
    canvas.itemconfig(predict_2, text=generate_prediction_text(prediction[1]))
    canvas.itemconfig(predict_3, text=generate_prediction_text(prediction[2]))
    canvas.itemconfig(predict_4, text=generate_prediction_text(prediction[3]))
    canvas.itemconfig(predict_time, text=generate_prediction_time(total_time))
    

def update_canvas_image(canvas, image_id, class_name_id):
    """
    Update the image for prediction
    
    """
    global global_image_path
    global new_image
    image_path, image_class = random_image_selector()
    global_image_path = image_path
    new_image = PhotoImage(file=image_path)
    canvas.itemconfig(image_id, image=new_image)
    canvas.itemconfig(class_name_id, text=image_class)
    
    
    
def create_class_display(canvas, x_position, y_position, model_name):
    """
    Display the prediction percentage for each model
    
    """
    x_difference = 150
    title_gap = 30
    text_gap = 20
    canvas_model_name = canvas.create_text(
        x_position, y_position, 
        text=model_name, 
        fill="black", 
        font=('Helvetica bold', 12)
    )
    canvas_model_prediction_1 = canvas.create_text(
        x_position - x_difference, y_position + title_gap, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 12),
        anchor='nw'
    )
    canvas_model_prediction_2 = canvas.create_text(
        x_position - x_difference, y_position + title_gap + text_gap, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 12),
    anchor='nw'
    )
    canvas_model_prediction_3 = canvas.create_text(
        x_position - x_difference, y_position + title_gap + text_gap * 2, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 12),
        anchor='nw'
    )
    canvas_model_prediction_4 = canvas.create_text(
        x_position - x_difference, y_position + title_gap + text_gap * 3, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 12),
        anchor='nw'
    )
    canvas_model_prediction_time = canvas.create_text(
        x_position - x_difference, y_position + title_gap + text_gap * 4, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 12),
        anchor='nw'
    )
    return [
        canvas_model_prediction_1,
        canvas_model_prediction_2,
        canvas_model_prediction_3,
        canvas_model_prediction_4,
        canvas_model_prediction_time
    ]
    

def create_gui():
    """
    Create a gui using tkinter and display outputs for MobileNet Large
    Small and Custom V5
    """
    window = tk.Tk()
    window.geometry("1000x800")
    
    canvas = Canvas(window, width = 1000, height = 800)      
    canvas_chosen_image = canvas.create_image(50, 300, anchor=NW, image=None) 
    canvas_text = canvas.create_text(500, 20, text="MobileNetCustom Demo", fill="black", font=('Helvetica bold', 20))
    
    mobile_large_id_list = create_class_display(canvas, 700, 100, "Mobile Large V3")
    mobile_small_id_list = create_class_display(canvas, 700, 300, "Mobile Small V3")
    mobile_custom_id_list = create_class_display(canvas, 700, 500, "Mobile Custom V3")
    
    canvas_class_name = canvas.create_text(
        160, 250, 
        text="", 
        fill="black", 
        font=('Helvetica bold', 18),
    )
    canvas.pack() 
    
    button_position = 700
    button_left_shift = 300
    
    button_new = Button(
        window, 
        text='Generate Prediction Large', 
        width=25,
        height=2, 
        bd='10', 
        command=lambda : update_canvas_image(canvas, canvas_chosen_image, canvas_class_name)
    )
    button_new.place(x=button_left_shift - 200, y=button_position)
    button_large = Button(
        window, 
        text='Generate Prediction Large', 
        width=25,
        height=2, 
        bd='10', 
        command=lambda : update_canvas(
            canvas, 
            mobile_large_id_list,
            model_large
        )
    )
    button_large.place(x=button_left_shift, y=button_position)
    button_small = Button(
        window, 
        text='Generate Prediction Small', 
        width=25,
        height=2, 
        bd='10', 
        command=lambda : update_canvas(
            canvas, 
            mobile_small_id_list,
            model_small
        )
    )
    button_small.place(x=button_left_shift + 200, y=button_position)
    button_custom = Button(
        window, 
        text='Generate Prediction Custom', 
        width=25,
        height=2, 
        bd='10', 
        command=lambda : update_canvas(
            canvas, 
            mobile_custom_id_list,
            model_custom
        )
    )
    button_custom.place(x=button_left_shift + 400, y=button_position)
    mainloop()
    
    
create_gui()

1/1 [==============================] - 0s 17ms/step
